In [1]:
import sys
sys.path.append('./src')
import warnings
warnings.filterwarnings("ignore")

from data import PPCI

In [2]:
# load dataset
encoder = "dino"
dataset = PPCI(encoder = encoder,
               token = "class",
               task = "or",
               split_criteria = "position_easy",
               environment = "supervised",
               batch_size = 64,
               num_proc = 4,
               verbose = True,
               data_dir = 'data/istant_hq',
               results_dir = f'results/istant_hq/{encoder}')

Embeddings from encoder 'dino' token 'class' already extracted for the supervised environment.
Training Environments:  [ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 40 41 42 43 44]
Validation Environments:  [35 36 37 38 39]
Prediction-Powered Causal Inference dataset successfully loaded.


In [3]:
# example train
dataset.train(add_pred_env="supervised", 
            hidden_layers = 1,
            hidden_nodes = 256,
            batch_size = 128,
            lr = 0.0005,
            num_epochs=5,
            save = False,
            verbose=True,
            multidomain=True,
            ic_weight=0,
            seed=1)
dataset.evaluate(color=None, verbose=False)

Device: cuda
Train Heterogenous, IC Weight: 0
Starting perfomances
  train:  Accuracy=0.778, Balanced Accuracy=0.500,  Precision=0.353, Recall=0.002
  val:  Accuracy=0.768, Balanced Accuracy=0.509,  Precision=0.778, Recall=0.020
Epoch 1 (train loss: 61.969)
  train:  Accuracy=0.331, Balanced Accuracy=0.570,  Precision=0.249, Recall=1.000
  val:  Accuracy=0.407, Balanced Accuracy=0.611,  Precision=0.284, Recall=0.999
Epoch 2 (train loss: 30.412)
  train:  Accuracy=0.699, Balanced Accuracy=0.796,  Precision=0.423, Recall=0.969
  val:  Accuracy=0.772, Balanced Accuracy=0.835,  Precision=0.508, Recall=0.955
Epoch 3 (train loss: 23.186)
  train:  Accuracy=0.816, Balanced Accuracy=0.860,  Precision=0.549, Recall=0.941
  val:  Accuracy=0.848, Balanced Accuracy=0.865,  Precision=0.623, Recall=0.897
Epoch 4 (train loss: 19.198)
  train:  Accuracy=0.885, Balanced Accuracy=0.887,  Precision=0.685, Recall=0.891
  val:  Accuracy=0.880, Balanced Accuracy=0.850,  Precision=0.723, Recall=0.794
Epoch 5

{'loss_val': 1.2698743343353271,
 'inv_loss_val': 0.16922955418360658,
 'acc_val': 0.8479999899864197,
 'bacc_val': 0.864840030670166,
 'TEB_val': 3242.785799920211,
 'acc': 0.8193181753158569,
 'bacc': 0.8606929183006287,
 'TEB': -0.02760230004787445,
 'TEB_bin': -0.024562492966651917,
 'EAD': 0.062312498688697815}